<a href="https://colab.research.google.com/github/forbeso/JaCovid/blob/develop/Jamaica_Covid_19_Cases_By_Parish.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import folium
from folium import FeatureGroup, LayerControl, Map, Marker, Choropleth
import json
import os
import pathlib as pl
from bs4 import BeautifulSoup
import urllib.request,json
import requests
from folium.plugins import MarkerCluster

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
#get data from api 

url ='https://admin.jamcovid19.moh.gov.jm/public/api/statistics?type=1'

r = requests.get(url)
dictr = r.json()
recs = dictr['data']['parishes_wise_report']
df = pd.json_normalize(recs)
#print(df)

frame = pd.DataFrame(data=df)

#select columns needed
ParishData = frame.loc[:,['parish','total_tests_positive','number_of_deaths','number_of_people_recovered']]

#rename columns
ParishData.rename(columns={'parish':'Parish',
                           'total_tests_positive':'Confirmed','number_of_deaths':'Deaths', 
                           'number_of_people_recovered':'Recoveries'}, inplace=True)
ParishData


In [0]:
#use folium to get Jamaica's Map
jamaica = folium.Map(
    location=[18.1096,-77.2975],
    zoom_start=9,
    tiles='Mapbox Bright'
)

In [0]:
ParishData.dtypes

In [0]:
#change data types
ParishData['Parish'] = ParishData['Parish'].astype(str)

ParishData['Confirmed'] = pd.to_numeric(ParishData['Confirmed'])
ParishData['Recoveries'] = pd.to_numeric(ParishData['Recoveries'])
ParishData['Deaths'] = pd.to_numeric(ParishData['Deaths'])

ParishData.dtypes

In [0]:
#get geojosn file with Jamaica's Parishes/Boundaries
geojson = os.path.join(pl.Path.cwd(),'data', '/content/drive/My Drive/Colab Notebooks/Data/JMParishes.json')
jamaica_geo = json.load(open(geojson))

In [0]:
#create Choropleth Map of Jamaica 
choro=Choropleth(
    geo_data = jamaica_geo,
    data = ParishData,
    name = 'choro',
    columns = ['Parish','Confirmed'],
    key_on='feature.id',
    fill_color='YlOrRd',
    legend_name = 'Confirmed Corona Virus Cases by Parish'
).add_to(jamaica)

In [0]:
# tiles = ['openstreetmap']

# for tile in tiles:
#     folium.TileLayer(tile).add_to(jamaica)

# folium.LayerControl().add_to(jamaica)

In [0]:
choro.geojson.add_child(
    folium.features.GeoJsonTooltip(['PARISH'], labels=False)
)


In [0]:
folium.CircleMarker(
    location=[jamaica_geo['features'][0]['geometry']['coordinates'][0][0]],
    radius=25,
    fill=True,
    popup=folium.Popup('inline explicit Popup')
).add_to(jamaica)

In [0]:
jamaica

In [0]:
jamaica.save('jamaica.html')